### Downloaded data 
https://www.kaggle.com/competitions/playground-series-s4e11/data
### Reading data 
train.csv from data folder 


In [15]:
import pandas as pd
import math
import os

# Load the dataset
df = pd.read_csv("../data/train.csv")

max_rows = len(df)
num_rows = int(25e3) 
total_chunks = math.ceil(max_rows / num_rows)

output_dir = "../data/raw"
os.makedirs(output_dir, exist_ok=True)

for idx in range(total_chunks):
    start = idx * num_rows 
    end = min(max_rows, (idx + 1) * num_rows)  
    
    part = df.iloc[start:end]

    target = os.path.join(output_dir, f"train-{idx}.csv")
    if end == max_rows: 
        target = os.path.join(output_dir, "test.csv")

    print(f"Generating rows {start} - {end - 1} into: {target}")
    part.to_csv(target, index=False)


Generating rows 0 - 24999 into: ../data/raw/train-0.csv
Generating rows 25000 - 49999 into: ../data/raw/train-1.csv
Generating rows 50000 - 74999 into: ../data/raw/train-2.csv
Generating rows 75000 - 99999 into: ../data/raw/train-3.csv
Generating rows 100000 - 124999 into: ../data/raw/train-4.csv
Generating rows 125000 - 140699 into: ../data/raw/test.csv
